In [39]:
import yfinance as yf
import yfinance.shared as shared
import time
import pandas as pd
# from datetime import date, timedelta, datetime
from myUtils import pickle_dump, pickle_load, read_symbols_file # NOQA
from myUtils import drop_symbols_all_NaN, chunked_list # NOQA
from myUtils import yf_download_AdjOHLCV_noAutoAdj
from yf_utils import _2_split_train_val_test, _3_random_slices, _4_perf_ranks

verbose = False  # True prints more output

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

filename_symbols = path_data_dump + 'vg_symbols_4chars_max.csv'  # symbols text file
filename_pickled_df_OHLCVA_downloaded = 'df_OHLCVA_downloaded '  # OHLCVA downloaded from Yahoo
filename_pickled_df_adjOHLCV = 'df_adjOHLCV'  # adjusted OHLCV
filename_pickled_df_symbols_close = "df_symbols_close"  # symbols' adjusted close
filename_pickled_symbols_df_adjOHLCV =  'symbols_df_adjOHLCV'  # symbols in df_adjOHLCV
filename_pickled_df_c = 'df_close_clean' 

In [40]:
df_c = pickle_load(path_data_dump, filename_pickled_df_c)
df_train, df_val, df_test = _2_split_train_val_test(df_c)
len_df_train = len(df_train)
len_df_val = len(df_val)
len_df_test = len(df_test)

In [41]:
# _n_samples = 2  # number of tuples to create for iloc start_train:end_train:end_eval
_n_samples = 2  # number of tuples to create for iloc start_train:end_train:end_eval
_days_lookbacks = [60, 120, 30]
_days_eval = 10

In [42]:
# create n_samples of iloc start_train:end_train:end_eval using max value in days_loobacks
max_train_eval_days = _3_random_slices(len_df_train, n_samples=_n_samples, days_lookback=max(_days_lookbacks), days_eval=_days_eval)
max_train_eval_days

[(156, 276, 286), (473, 593, 603)]

In [ ]:
def lookback_slices(max_slices, days_lookbacks, verbose=False):
  """Create 
  
  
  Returns perf_ranks_dict(dic. of dic. of symbols ranked in descending
    performance) and ranked_perf_ranks_dict(dic. of symbols ranked in
    descending frequency in a combined pool of symbols in perf_ranks_dict).

  Args:
      df_close(dataframe): dataframe of symbols' close with
        DatetimeIndex e.g. (['2016-12-19', ... '2016-12-22']), symbols as
        column names, and symbols' close as column values.
      days_lookbacks(list of positive integers): list of number of days to
      look-back, e.g. [15, 30], for performance calculation.
      n_top_syms(int): number of top symbols to keep in perf_ranks_dict 

  Return:
      perf_ranks_dict({dic): dic. of dic. of symbols ranked in descending
        performance.
        First dic keys are:
        'period' + str(days_lookbacks[0]), ... ,
        'period' + str(days_lookbacks[-1])
        Second dic keys are:
        'r_CAGR/UI', 'r_CAGR/retnStd' and 'r_retnStd/UI'
        e.g.:
        {
          period-15': {
                        'r_CAGR/UI':  ['HZNP', ... , 'CB'],
                        'r_CAGR/retnStd': ['BBW', ... , 'CPRX'],
                        'r_retnStd/UI':   ['ENR', ... , 'HSY']
                      },
          ... ,
          'period-60': {
                        'r_CAGR/UI':  ['WNC', ... , 'FSLR'],
                        'r_CAGR/retnStd': ['VCYT', ... , 'BERY'],
                        'r_retnStd/UI':   ['MYOV', ... , 'NSC']
                        }
        }
      ranked_perf_ranks_dict(dic): dic. of symbols ranked in descending
        frequency in a combined pool of symbols in perf_ranks_dict.  Key is
        'ranked_perf_ranks_period' + str(days_lookbacks), e.g.:
        {'ranked_perf_ranks_period[-15, -30]': ['HZNP', ... , 'NSC']}
  """  
  lb_slices = []
  days_lookbacks.sort()  # sort list of integers in ascending order
  for l_max_slice in max_slices:
    for days in days_lookbacks:
      new_slice = (l_max_slice[1]-days, l_max_slice[1], l_max_slice[2])
      lb_slices.append(new_slice)
      if verbose:
        print(f'days: {days}, {new_slice}')      
    if verbose:    
     print('')
  return lb_slices

In [43]:
def lookback_slices_2(max_slices, days_lookbacks, verbose=False):

  lb_slices = []
  days_lookbacks.sort()  # sort list of integers in ascending order
  for max_slice in max_slices:
    l_max_slice = []
    for days in days_lookbacks:
      new_slice = (max_slice[1]-days, max_slice[1], max_slice[2])
      l_max_slice.append(new_slice)
      if verbose:
        print(f'days: {days}, {new_slice}')    
    lb_slices.append(l_max_slice)  
    if verbose:    
     print('')
  return lb_slices

In [44]:
lb_slices = lookback_slices_2(max_slices=max_train_eval_days, days_lookbacks=_days_lookbacks, verbose=False)
lb_slices 

[[(246, 276, 286), (216, 276, 286), (156, 276, 286)],
 [(563, 593, 603), (533, 593, 603), (473, 593, 603)]]

In [ ]:
grp_perf_ranks
# grp_most_common_syms

In [49]:
# grp_perf_ranks = {}
grp_most_common_syms = []
# loop thru lists of tuples of start_train:end_train:end_eval, i.e.
#  [[(887, 917, 927), (857, 917, 927), (797, 917, 927)],
#  [(483, 513, 523), (453, 513, 523), (393, 513, 523)]]
for l_max_slices in lb_slices:  
  for l_max_slice in l_max_slices:  # l_max_slice, e.g. (246, 276, 286)
    start_train = l_max_slice[0]
    end_train = l_max_slice[1]
    start_eval = end_train
    end_eval = l_max_slice[2]
    days_lookback = end_train - start_train
    days_eval = end_eval - start_eval
    print(f'l_max_slices:  {l_max_slices}')
    print(f'l_max_slice:   {l_max_slice}')
    print(f'days_lookback: {days_lookback}')
    print(f'days_eval:     {days_eval}')    
    print(f'start_train:   {start_train}')
    print(f'end_train:     {end_train}')
    # print(f'start_eval:    {start_eval}')
    # print(f'end_eval:      {end_eval}')

    _df = df_train.iloc[start_train:end_train]
    perf_ranks, most_common_syms = _4_perf_ranks(_df, n_top_syms=10)
    # 1 days_lookback of r_CAGR/UI, r_CAGR/retnStd, r_retnStd/UI
    print(f'perf_ranks: {perf_ranks}')  
    # most common symbols of perf_ranks 
    print(f'most_common_syms: {most_common_syms}')     
    # grp_perf_ranks[days_lookback] = perf_ranks
    print('+++ finish 1 days_lookback +++', '\n')
    grp_most_common_syms.append(most_common_syms)
    
  print(f'grp_most_common_syms: {grp_most_common_syms}')
  print('===== finish 1 set of days_lookbacks =====', '\n')

l_max_slices:  [(246, 276, 286), (216, 276, 286), (156, 276, 286)]
l_max_slice:   (246, 276, 286)
days_lookback: 30
days_eval:     10
start_train:   246
end_train:     276
perf_ranks: {'period-30': {'r_CAGR/UI': array(['VIRT', 'BEEM', 'BXC', 'NVAX', 'MRTX', 'KURA', 'WK', 'WING',
       'LQDT', 'VSTO'], dtype=object), 'r_CAGR/retnStd': array(['BEEM', 'BXC', 'NVAX', 'KURA', 'MRTX', 'VIRT', 'VSTO', 'LQDT',
       'ASND', 'COLL'], dtype=object), 'r_retnStd/UI': array(['VIRT', 'WK', 'KDP', 'WING', 'RM', 'BXC', 'TTGT', 'SII', 'DECK',
       'MTCH'], dtype=object)}}
most_common_syms: [('VIRT', 3), ('BXC', 3), ('BEEM', 2), ('NVAX', 2), ('MRTX', 2), ('KURA', 2), ('WK', 2), ('WING', 2), ('LQDT', 2), ('VSTO', 2), ('ASND', 1), ('COLL', 1), ('KDP', 1), ('RM', 1), ('TTGT', 1), ('SII', 1), ('DECK', 1), ('MTCH', 1)]
+++ finish 1 days_lookback +++ 

l_max_slices:  [(246, 276, 286), (216, 276, 286), (156, 276, 286)]
l_max_slice:   (216, 276, 286)
days_lookback: 60
days_eval:     10
start_train:   216
en

In [ ]:
grp_perf_ranks = {}
grp_most_common_syms = {}

for lb_slice in lb_slices:
  start_train = lb_slice[0]
  end_train = lb_slice[1]
  start_eval = end_train
  end_eval = lb_slice[2]
  days_lookback = end_train - start_train
  days_eval = end_eval - start_eval
  print(f'lb_slice:      {lb_slice}')
  print(f'days_lookback: {days_lookback}')
  print(f'days_eval:     {days_eval}')    
  print(f'start_train:   {start_train}')
  print(f'end_train:     {end_train}')
  # print(f'start_eval:    {start_eval}')
  # print(f'end_eval:      {end_eval}')
  print('')
  _df = df_train.iloc[start_train:end_train]
  perf_ranks, most_common_syms = _4_perf_ranks(_df, n_top_syms=100)
  grp_perf_ranks[days_lookback] = perf_ranks
  grp_most_common_syms[days_lookback] = most_common_syms
  print(f'perf_ranks: {perf_ranks}')
  print(f'most_common_syms: {most_common_syms}')
  print(f'grp_perf_ranks: {grp_perf_ranks}')
  print(f'grp_most_common_syms: {grp_most_common_syms}')
  print('====', '\n')     